<a href="https://colab.research.google.com/github/zzhining/stock_market_analysis/blob/main/4%EC%A3%BC%EC%B0%A8/10_paxnet_%EC%A6%9D%EA%B6%8C%EC%82%AC_%EC%A2%85%EB%AA%A9_%EB%A6%AC%ED%8F%AC%ED%8A%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 파이썬을 이용한 리포트 수집

In [2]:
import requests
from bs4 import BeautifulSoup as BS

In [3]:
url = "http://www.paxnet.co.kr/stock/report/report?wlog_rpt=jm&menuCode=2222"
response = requests.get(url)
html = response.content
html = BS(html)

In [4]:
div = html.find("div", attrs = {"class":"board-type"})
li_list = div.find_all("li")

#### 수집 대상 행 결정

In [5]:
print(li_list[0])

<li class="board-list-th">
<div>종목</div>
<div>제목</div>
<div class="right">적정가격</div>
<div>투자의견</div>
<div>제공출처</div>
<div class="cent">작성일</div>
</li>


In [6]:
print(li_list[1])

<li>
<div><strong class="color-cate"><span class="bracket">[</span><a href="http://www.paxnet.co.kr/stock/analysis/main?wlog_rpt=list_jm&amp;abbrSymbol=205100">엑셈</a><span class="bracket">]</span></strong></div>
<div>
<p>
<a href="javascript:selectView('145087');">AI와 빅데이터로 혁신을 이끄는 소프트웨어 공급사</a>
</p>
</div>
<div class="line3">
<span>적정가격</span>
	 								
										
										
											-원
										
									
								</div>
<div class="line3 color-black">
									-
								</div>
<div class="line3">한국IR협의회</div>
<div class="line3">2024.05.21</div>
</li>


In [7]:
print(li_list[-1])

<li><a href="#" onclick="linkPage(5); return false;">5</a></li>


In [8]:
check_str = '<strong class="color-cate"><span class="bracket">'

In [9]:
print(check_str in str(li_list[1]))
print(check_str in str(li_list[0]))
print(check_str in str(li_list[-1]))

True
False
False


#### 파싱 전략 수립

In [10]:
stock = li_list[1].find_all("a")[0].text
print(stock)

엑셈


In [11]:
title = li_list[1].find_all("a")[1].text
print(title)

AI와 빅데이터로 혁신을 이끄는 소프트웨어 공급사


In [12]:
price = li_list[1].find("div", attrs = {"class":"line3"}).text
display(price)

'\n적정가격\r\n\t \t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\t\t-원\r\n\t\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t\t\r\n\t\t\t\t\t\t\t\t'

In [13]:
price = price.replace('\r', '').replace('\t', '').replace('\n', '')
price = price.split(' ')[1].replace(',', '')[:-1]
display(price)

'-'

In [14]:
opinion = li_list[1].find_all("div", attrs = {"class":"line3"})[1].text
display(opinion)

'\r\n\t\t\t\t\t\t\t\t\t-\r\n\t\t\t\t\t\t\t\t'

In [15]:
opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
display(opinion)

'-'

In [16]:
trading_firm = li_list[1].find_all("div", attrs = {"class":"line3"})[2].text
display(trading_firm)

'한국IR협의회'

In [17]:
date = li_list[1].find_all("div", attrs = {"class":"line3"})[3].text
display(date)

'2024.05.21'

In [18]:
import pandas as pd
date = pd.to_datetime(date)
display(date)

Timestamp('2024-05-21 00:00:00')

#### 단일 리포트 파싱

In [19]:
def parsing_li(li):
    stock = li.find_all("a")[0].text # 종목명
    title = li.find_all("a")[1].text # 리포트 제목
    price = li.find("div", attrs = {"class":"line3"}).text # 적정가격
    price = price.replace('\r', '').replace('\t', '').replace('\n', '')
    price = price.split(' ')[1].replace(',', '')[:-1]
    opinion = li.find_all("div", attrs = {"class":"line3"})[1].text # 의견
    opinion = opinion.replace('\t', '').replace('\n', '').replace('\r', '')
    trading_firm = li.find_all("div", attrs = {"class":"line3"})[2].text # 증권사
    date = li.find_all("div", attrs = {"class":"line3"})[3].text # 날짜
    date = pd.to_datetime(date)

    return [stock, title, price, opinion, trading_firm, date]

In [20]:
print(parsing_li(li_list[1]))

['엑셈', 'AI와 빅데이터로 혁신을 이끄는 소프트웨어 공급사', '-', '-', '한국IR협의회', Timestamp('2024-05-21 00:00:00')]


#### 한 페이지 내 모든 리포트 수집 및 정제

In [21]:
data = []
for li in li_list:
    if check_str in str(li):
        record = parsing_li(li)
        data.append(record)
data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])

In [22]:
display(data.head(10))

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,엑셈,AI와 빅데이터로 혁신을 이끄는 소프트웨어 공급사,-,-,한국IR협의회,2024-05-21
1,동양생명,Q24 Review: CSM 성장 지속,6400,매수,메리츠증권,2024-05-21
2,삼성생명,1Q24 Review: 구체화될 방향성,110000,매수,메리츠증권,2024-05-21
3,GKL,드롭액 개선세 지속,19000,매수,유안타증권,2024-05-21
4,메가스터디,#의대정원 확대 #주주환원 #밸류에이션,93000,매수,유안타증권,2024-05-21
5,코오롱인더,"2024년 2 ~ 3분기, 3가지 변화에 주목!",70000,매수,유안타증권,2024-05-21
6,파인엠텍,Anything is ok!,-,-,유안타증권,2024-05-21
7,제이티,하반기를 기다린다,12000,매수,IBK투자증권,2024-05-21
8,비즈니스온,Q24 Review: 역대 분기 최대 실적 달성 성공,-,매수,유진투자증권,2024-05-21
9,한화생명,1Q24 Review: 체질 개선 진행 중,4000,매수,메리츠증권,2024-05-17


In [23]:
import time
base_url = "http://www.paxnet.co.kr/stock/report/report?menuCode=2222&currentPageNo={}&reportId=0&searchKey=stock&searchValue="
data = []

for page_no in range(1, 563):
    # 웹 페이지 소스 가져오기
    url = base_url.format(page_no)
    while True:
        response = requests.get(url)
        if response.status_code == 200:
            time.sleep(1)
            break
        else:
            time.sleep(10 * 60)
    html = response.content
    html = BS(html)

    # 웹 페이지 소스에서 필요한 부분만 가져오기
    div = html.find("div", attrs = {"class":"board-type"})
    li_list = div.find_all("li")
    for li in li_list:
        if check_str in str(li):
            record = parsing_li(li)
            data.append(record)

data = pd.DataFrame(data, columns = ["종목명", "리포트제목", "적정가격", "의견", "증권사", "날짜"])

In [24]:
data

,종목명,리포트제목,적정가격,의견,증권사,날짜
0,엑셈,AI와 빅데이터로 혁신을 이끄는 소프트웨어 공급사,-,-,한국IR협의회,2024-05-21
1,동양생명,Q24 Review: CSM 성장 지속,6400,매수,메리츠증권,2024-05-21
2,삼성생명,1Q24 Review: 구체화될 방향성,110000,매수,메리츠증권,2024-05-21
3,GKL,드롭액 개선세 지속,19000,매수,유안타증권,2024-05-21
4,메가스터디,#의대정원 확대 #주주환원 #밸류에이션,93000,매수,유안타증권,2024-05-21
...,...,...,...,...,...,...
16855,농심,2Q18 Preview: 시장기대치를 하회하는 영업실적 예상,400000,매수(유지),HI투자증권,2018-07-04
16856,오뚜기,2Q18 Preview: 시장기대치에 부합하는 영업실적 예상,1100000,매수(유지),HI투자증권,2018-07-04
16857,오리온,2Q18 Preview: 시장기대치에 부합하는 영업실적 예상,180000,매수(유지),HI투자증권,2018-07-04
16858,CJ제일제당,2Q18 Preview:시장기대치에 부합하는 영업실적 예상,460000,매수(유지),HI투자증권,2018-07-04


In [ ]:
data.to_csv("종목리포트.csv")